# List of words to replace

In [3]:
unemployed_list = ['UNEMPLOYED', 'Unemployeed', 'Unemployment', 'Unemployment Insurance', 'UNEMPLOYED-','Un-Employed', 'Unemployed Sound Designer','Unemployment Comp Ex', 'Not Employ', 'Not Employed', 'Not Employeed','Not-Employed', 'Unemployed (Fired When I Had To Quaran','Unemployed Engineer','Unemployment Tax Agent','NOT EMPLOYED', 'Not Employed-Student', 'NOT MUCH', 'Not-Employed', 'Unemployed/Disabled', 'Not', 'Not Employed - Student', 'Not Employed At The Moment', 'Not Employed1', ' unempoyed','None Yo', 'Nonooo', "None",'NOT EMPLOYED', 'Not Employed :(', 'Unemployed :(', 'Not Unemloyed', 'NOT WORKING', 'Not Emploted', 'Not Employ', 'Not Emplyed', 'Not EmployedOccidental College', 'Not Employee D', 'Not Emplyed', 'Not+Employed', 'Nose Picker', 'Not mployed', 'Not. Employed', 'Noyb', 'No', 'No Job', 'Not Employedi', 'Not Enplyed', 'Not Working', 'Not Employes', 'Not EmployLawyer', 'Not Empoyeed', 'Not ESocial Worker', 'Note Employed', 'Not Employed (Stay At Home Dad)']

retired_list = ['Retired',
'RETIRED',
'RETIRED-34 YRS BANKING',
'Dist Mgr--Retired',
'Retired',
'Not Employed Retired',
'Bellman/Retired Electric Utility Mid-L',
'Instructor--Retired',
'Nurse Practitioner (Retired)',
'Retired/Teacher',
'Physician/Retired',
'Attorney - Retired',
'RETIRED/',
'Not Employed-Retired',
'RET.',
'Instructor--Retired',
'Disabled/Retired',
'Retired,',
'Retired(Manager,Leg Consultant, Social',
'(Ret)',
'RET',
'Retired-Chemist',
'Founder/Retired',
'Retired/Self-Employed',
'RANCHER/RET',
'Retired/Disabled',
'Retireed',
'Not Employed/Retired',
'(RETIRED)',
'Farmer/Retired Attorney',
'Intern/Retired RN',
'(Retired)',
'Retiredretired',
'Retired/Hotelier',
'Tutor/Retired Teacher And Administrato',
'R-Sales C-Retired',
'Retiredrrtired',
'Retired.',
'Surgeon/Retired/Professor Emeritus',
'ELECTRONICS TECH NAVY RET',
'(Retiree)',
'Retireded',
'Retired/Recording Secretary',
'Retire',
'RETEIRED',
'RETIED',
"Retired’",
'RETIURED',
'RETIERD',
'Retiered'
'Ret’d',
'Retired/Epidemiologist',
'Retied',
'Retired/Volunteer',
'Retred',
'RETRIED',
'RetiredJudge/Mediator',
'RETIRED.',
'Retired!',
'Retiree',
'Retired/Part-Time Consultant',
'RETORED',
'Retirednone'
'RETRED',
'Retiref',
'Retried',
'RETEIRED',
'RETIREDRETIRED',
'RETRED',
'Artist/Retired',
'%Retired']

republican_campaigns = ['Donald J. Trump For President, Inc.', 'Weld 2020 Presidential Campaign Committee, Inc.']

In [2]:
#Change this to config file.

user = "postgres"
password = "YAhabibi1!"
host = "localhost"
port = "5432"
db = "Election_db"
uri = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}"


# Raw data import 

In [1]:
#Libraries neeeded
from sqlalchemy import create_engine
import pandas as pd
import sys

#Creates engine to sqlite.
engine_lite = create_engine("sqlite:///Election2020Data.sqlite")

#Stablishes connection.
conn_lite = engine_lite.connect()

#Holder for the dataframes
dataFrames = []

#For chunks in sql table. Chunk size = size per table. Index_col = column for index. columns = list of columns to import from database.
for chunks in pd.read_sql_table("2020Donors",conn_lite,chunksize=500000, index_col = 'Record_id', columns=['Record_id','Flag_Orgind','City','State','Zip','Occupation','Amount','Date','Aggregate_Amount','Cycle','Campaign']):
    
    #Appends each dataframe chunk into a list.
    dataFrames.append(chunks)


dataFrames[0].head()

#Close connection and dispose engine.
conn_lite.close()
engine_lite.dispose()

,Flag_Orgind,City,State,Zip,Occupation,Amount,Date,Aggregate_Amount,Cycle,Campaign
Record_id,,,,,,,,,,
1,IND,Birmingham,AL,35205,Retired,250.0,2019-03-31,250.00,2020,Kamala Harris For The People
2,IND,Mobile,AL,36695,Clerk,100.0,2019-03-31,250.00,2020,Kamala Harris For The People
3,IND,Mountain Brk,AL,35213,Retired,500.0,2019-03-31,500.00,2020,Kamala Harris For The People
4,IND,Birmingham,AL,35242,PT Consultant,25.0,2019-03-30,297.84,2020,Kamala Harris For The People
5,IND,Birmingham,AL,35242,PT Consultant,25.0,2019-03-28,297.84,2020,Kamala Harris For The People


# Data cleaning.

In [27]:
#Empty holder for clean dataframes. -------------------------------------
import csv

cleanedDataFrames = []

with open('combinedClean.csv', 'w', newline='') as csvfile: #Clean csv to upload to S3 
    
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['City','State','Zip','Occupation','Amount','Date','Aggregate_Amount','Cycle','Campaign'])
    
    #for dataframe in dataframes list.

    for dataframe in dataFrames:

        dataframe['Occupation'] = dataframe['Occupation'].str.title()

        #add here all the changes you want to do. ------------------------
        filteredDataframe = dataframe[dataframe['Flag_Orgind'] == 'IND']

        for index, row in filteredDataframe.iterrows():

            #Occupation filter.
            occupationTest = False
            try:
                for word in retired_list:
                    if word in row['Occupation']:
                        row['Occupation'] = 'Retired'
                        occupationTest = True 

                if (row['Occupation'] in unemployed_list) & (occupationTest == False) :
                    row['Occupation'] = 'Unemployed'
                    occupationTest = True

                if occupationTest == False: 
                    row['Occupation'] = 'Employed'

            except TypeError: #People are willing to type unemployed or retired becasue they want to whine or are proud of retiring. Employed people tend to not disclose
                row['Occupation'] = 'Employed'

            if (row['Campaign'] == 'Donald J. Trump For President, Inc.') | (row['Campaign'] == 'Weld 2020 Presidential Campaign Committee, Inc.'):
                row['Campaign'] = 'Republican'
            else:
                row['Campaign'] = 'Democrat'
    
            csvwriter.writerow([row['City'],row['State'],row['Zip'],row['Occupation'],row['Amount'],pd.to_datetime(row['Date']),row['Aggregate_Amount'],row['Cycle'],row['Campaign']])

